Here are customized functions to detect transmission events on a subset of samples on which you run StrainPhlAn

**This is only compatible with MetaPhlAn (StrainPhlAn) 4.0.6** \
**Make sure you replace "[YOUR CONDA ENVIRONMENT FOLDER]" in your function with the path of your conda environment with MetaPhlAn 4.0.6 installed**

How it works:
For example, your goal is to detect transmission events for clade SGB1877 on a subset of your samples, make sure you have run StrainPhlAn for SGB1877
1. First you run `extract_seq` function to extract the subset samples of interest from "[Your StrainPhlAn output directory]/SGB1877/SGB1877.StrainPhlAn4_concatenated.aln", the filtered fasta will be "[Your StrainPhlAn output directory]/SGB1877/SGB1877.filtered.fasta"
2. Then, you can run `run_raxml` to make phylogenetic inference on the filtered fasta
3. Before running the transmission detection function, replace the 'strain_transmission.py' in the conda environment with the customized 'strain_transmission.py' file. This is to set 0.1 or (0.01, 0.05) threshold for clades that were not in VALLES 2023 PAPER [PMID: 36653448]
4. Run the `detect_transmission` function to detect transmission event 

In [1]:
from Bio import SeqIO

import subprocess
import os

ModuleNotFoundError: No module named 'Bio'

In [2]:
def extract_seq(strain_clade, sample_list,
                strainphlan_dir=os.path.abspath(strain_raw_dir),
                output_dir=os.path.abspath(strain_ana)):
    """
    Extract sequences of input clade from strainphlan alignment
    --------------------------------------------------
    strain_clade: str
        strain clade name
    sample_list: list
        list of samples included
    strainphlan_dir: str
        strainphlan raw data folder
    output_dir: str
        output folder to store the clade folders for the filtered fasta
    
    """

    # set up the path for the strainphlan alignment of the clade
    strainphlan_fasta_path = f'{strainphlan_dir}/{strain_clade}/{strain_clade}.StrainPhlAn4_concatenated.aln'

    # create the folder for output clade folder of the filtered fasta
    output_clade_dir = f'{output_dir}/{strain_clade}'

    if not os.path.exists(output_clade_dir):
        os.makedirs(output_clade_dir, exist_ok=True)

    output_fasta_path = f'{output_clade_dir}/{strain_clade}.filtered.fasta'

    # filter fasta files

    with open(strainphlan_fasta_path,'r') as input_fasta, open(output_fasta_path, 'w') as output_fasta:
        for record in SeqIO.parse(input_fasta, 'fasta'):
            if record.id in sample_list:
                SeqIO.write(record, output_fasta, 'fasta')

NameError: name 'os' is not defined

In [ ]:
# function for run RAxML

def run_raxml(strain_clade, strain_out_dir=os.path.abspath(strain_ana)):
    """
    run raxml on filtered fasta, THIS CAN ONLY BE RUN ON BIOBAKERY ENVIRONMENT ON MERGE
    --------------------------------------------------
    strain_clade: str
        strain clade name
    strain_out_dir: str
        folder to store the clade folders for the filtered fasta
    
    """
    
    # set up the command for raxml
    raxml_cmd = (f'[YOUR CONDA ENVIRONMENT FOLDER]/bin/raxmlHPC-PTHREADS-SSE3 -p 1989 -m GTRCAT -T 8 '
                 f'-w {strain_out_dir}/{strain_clade} '
                 f'-s {strain_out_dir}/{strain_clade}/{strain_clade}.filtered.fasta '
                 f'-n {strain_clade}.raxml.tre')
    # split the command to small trunks of strings
    raxml_cmd_list = raxml_cmd.split(' ')

    # run the command
    subprocess.run(raxml_cmd_list, shell=False)

## RUN THE BLOCK BELOW IF WANNA USE 0.1 THRESHOLD FOR STRAINS NOT IN VALLES 2022 PAPER

In [30]:
!cp strain_transmission.py [YOUR CONDA ENVIRONMENT FOLDER]/lib/python3.9/site-packages/metaphlan/utils/strain_transmission.py

In [31]:
def detect_transmission(strain_clade,
                        strain_out_dir=os.path.abspath(strain_ana),
                        transmission_meta_file=f'{os.path.abspath(strain_ana)}/strain_transmission_metadata.tsv'):
    """
    run transmission detection on the clade, output the return code
    THIS CAN ONLY BE RUN ON BIOBAKERY ENVIRONMENT ON MERGE
    --------------------------------------------------
    strain_clade: str
        strain clade name
    strain_out_dir: str
        folder to store the clade folders for the filtered fasta
    transmission_meta_file: str
        the metadata for transmission
    
    """
    # set up the command for transmission detection
    # if b.longum subspecies, use t__SGB17248
    if strain_clade in ['t__subsp.infantis', 't__subsp.longum']:
        str_trans_cmd = (f'[YOUR CONDA ENVIRONMENT FOLDER]/bin/strain_transmission.py '
                 f'-t {strain_out_dir}/{strain_clade}/RAxML_result.{strain_clade}.raxml.tre '
                 f'-m {transmission_meta_file} '
                 f'-o {strain_out_dir}/{strain_clade} '
                 f'--sgb_id t__SGB17248')
    else:
        str_trans_cmd = (f'[YOUR CONDA ENVIRONMENT FOLDER]/bin/strain_transmission.py '
                 f'-t {strain_out_dir}/{strain_clade}/RAxML_result.{strain_clade}.raxml.tre '
                 f'-m {transmission_meta_file} '
                 f'-o {strain_out_dir}/{strain_clade} '
                 f'--sgb_id {strain_clade}')

    str_trans_cmd_list = str_trans_cmd.split(' ')
    run_td = subprocess.run(str_trans_cmd_list, shell=False, stderr=True)

    return run_td.returncode